This file shows the parsing of the powerlogs and creating the figures presented in the paper.

The parsing of the powerlogs is inspired by .., as well as creating the graphs.

## Load libraries

In [2]:
import os
import subprocess
import re
import csv
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
import pandas as pd
import numpy as np
import time
import plotly.figure_factory as ff
from PIL import Image

## Parsing the Powermetrics Logs

In [3]:
def regexParse(content, model):
    # Declare local dataframes
    dfPower, dfFrequency, dfUsage = pd.DataFrame(), pd.DataFrame(), pd.DataFrame()

    # Efficiency Cluster Power, Frequency, and Usage
    dfPower['Efficiency Cluster'] = pd.Series(map(int, re.findall(r'E-Cluster HW active frequency:\s*([\d]+)\s*MHz', content)))
    dfFrequency['Efficiency Cluster'] = pd.Series(map(float, re.findall(r'E-Cluster HW active residency:\s*([\d.]+)%', content)))

    # Performance Cluster Power, Frequency, and Usage
    dfPower['Performance Cluster'] = pd.Series(map(int, re.findall(r'P-Cluster HW active frequency:\s*([\d]+)\s*MHz', content)))
    dfFrequency['Performance Cluster'] = pd.Series(map(float, re.findall(r'P-Cluster HW active residency:\s*([\d.]+)%', content)))

    # DRAM, Cluster, and Package Power
    dfPower['CPU Power'] = pd.Series(map(int, re.findall(r'CPU Power:\s*([\d.]+)\s*mW', content)))
    dfPower['GPU Power'] = pd.Series(map(int, re.findall(r'GPU Power:\s*([\d.]+)\s*mW', content)))
    dfPower['ANE Power'] = pd.Series(map(int, re.findall(r'ANE Power:\s*([\d.]+)\s*mW', content)))
    dfPower['Combined Power'] = pd.Series(map(int, re.findall(r'Combined Power \(CPU \+ GPU \+ ANE\):\s*([\d.]+)\s*mW', content)))

    # GPU Frequency and Usage
    dfFrequency['GPU'] = pd.Series(map(int, re.findall(r'GPU HW active frequency:\s*([\d]+)\s*MHz', content)))
    dfUsage['GPU'] = pd.Series(map(float, re.findall(r'GPU HW active residency:\s*([\d.]+)%', content)))

    # Ensure all data points have the same length
    min_length = min(len(dfPower), len(dfFrequency), len(dfUsage))
    dfPower = dfPower[:min_length]
    dfFrequency = dfFrequency[:min_length]
    dfUsage = dfUsage[:min_length]

    # Create time and videoType columns
    dataPoints = len(dfPower)
    dfPower['time'] = dfFrequency['time'] = dfUsage['time'] = list(range(1, dataPoints + 1))
    dfPower['Model'] = dfFrequency['Model'] = dfUsage['Model'] = [model] * dataPoints

    return dfPower, dfFrequency, dfUsage

In [6]:
directory_path = os.getcwd()

# Current directory should have a folder named powermetric-logs which contains the output logs of powermetric runs
powerLogsFolderName = "../data"

# Build the full path to the logs folder
pathLogsFolder = directory_path + '/' + powerLogsFolderName + '/'

# Get the list of all log files in the logs folder
powerLogsList = os.listdir(pathLogsFolder)

# Create local dataframes
dfPower, dfFrequency, dfUsage = pd.DataFrame(), pd.DataFrame(), pd.DataFrame()

# Parse each log file
for logsFile in powerLogsList:

    if not os.path.isfile(pathLogsFolder + logsFile): 
        print('File does not exist.')
    else:
        file = open(pathLogsFolder + logsFile, 'r', encoding="utf8", errors='ignore')
        content = file.read()
    
    if (logsFile.find('mp4') >= 0) or (logsFile.find('webm') >= 0):
        # Transform 4K-AV1.mp4.txt -> 4K-AV1 because that's what we want in the charts
        f_name = os.path.splitext(logsFile)[0]
        videoType = str.split(f_name, '.')[0]
    else:
        # Used for file paths like Safari-VP9-HW.txt i.e. without the video container (mp4, webm)
        videoType = os.path.splitext(logsFile)[0]

    # Parse the content and build Data Frames
    dfPowerTemp, dfFrequencyTemp, dfUsageTemp = regexParse(content, videoType)
    dfPower     = pd.concat([dfPower, dfPowerTemp], ignore_index=True)
    dfFrequency = pd.concat([dfFrequency, dfFrequencyTemp], ignore_index=True)
    dfUsage     = pd.concat([dfUsage, dfUsageTemp], ignore_index=True)

dfPower[['Model', 'Output Length', 'Input Length']] = dfPower['Model'].str.extract(r'(\w+)-(\d+)-(short|long)')


print(dfPower.head())
print(dfFrequency.head())
print(dfUsage.head())

   Efficiency Cluster  Performance Cluster  CPU Power  GPU Power  ANE Power  \
0                1287                  747        153         24          0   
1                1050                  611         39         24          0   
2                1094                  652         86          6          0   
3                1227                  618        106          6          0   
4                1052                  608         36         24          0   

   Combined Power  time   Model Output Length Input Length  
0             177     1  FlanT5           100         long  
1              45     2  FlanT5           100         long  
2             110     3  FlanT5           100         long  
3             112     4  FlanT5           100         long  
4              42     5  FlanT5           100         long  
   Efficiency Cluster  Performance Cluster  GPU  time            Model
0               55.80                 5.21  707     1  FlanT5-100-long
1               2

In [9]:
# Group by 'Model', 'Output Length', and 'Input Length' and calculate the mean of 'Combined Power' and 'time'
combined_power_time_summary = dfPower.groupby(['Model', 'Output Length', 'Input Length']).agg(
    {'Combined Power': 'mean', 'time': 'mean'}).reset_index()

# Calculate energy in joules
combined_power_time_summary['Energy (J)'] = (combined_power_time_summary['Combined Power'] / 1000) * combined_power_time_summary['time']

# Print the results
for _, row in combined_power_time_summary.iterrows():
    print(f"Model: {row['Model']}, Output Length: {row['Output Length']}, Input Length: {row['Input Length']}, "
          f"Average Combined Power: {row['Combined Power']:.2f} mW, Average Time: {row['time']:.2f} seconds, "
          f"Energy Consumption: {row['Energy (J)']:.4f} J")

# Print the summary DataFrame
print(combined_power_time_summary)


Model: Bloom, Output Length: 100, Input Length: long, Average Combined Power: 1982.31 mW, Average Time: 708.50 seconds, Energy Consumption: 1404.4642 J
Model: Bloom, Output Length: 100, Input Length: short, Average Combined Power: 2003.94 mW, Average Time: 695.50 seconds, Energy Consumption: 1393.7385 J
Model: Bloom, Output Length: 200, Input Length: long, Average Combined Power: 2182.14 mW, Average Time: 1631.00 seconds, Energy Consumption: 3559.0641 J
Model: Bloom, Output Length: 200, Input Length: short, Average Combined Power: 2183.55 mW, Average Time: 1554.50 seconds, Energy Consumption: 3394.3338 J
Model: Bloom, Output Length: 50, Input Length: long, Average Combined Power: 1811.27 mW, Average Time: 375.50 seconds, Energy Consumption: 680.1306 J
Model: Bloom, Output Length: 50, Input Length: short, Average Combined Power: 1786.67 mW, Average Time: 361.50 seconds, Energy Consumption: 645.8823 J
Model: FlanT5, Output Length: 100, Input Length: long, Average Combined Power: 1465.61 

## Plots

In [12]:
bgcolor = 'white' # white  181F26
titlecolor = '#707070' # 4c4943
graycolor = 'gray'
green = '#66D8BA'

In [75]:
def plot_average_power_bar(dataframe, title):
    # Calculate the average Combined Power for each model
    barDf = dataframe.groupby('Model')['Combined Power'].mean().reset_index()

    # Create a horizontal bar chart
    fig = px.bar(barDf, y='Model', x='Combined Power', template='plotly_white',
                 orientation='h', hover_name='Model', 
                 width=700, height=250, 
                 color_discrete_sequence=[green],
                 labels={"Combined Power": "Power Consumption (mW)"})

    fig.update_xaxes(zeroline=True, title_font=dict(size=12), color=titlecolor,
                     title_font_color=titlecolor, tickfont=dict(size=9), 
                     gridcolor=graycolor, zerolinecolor=graycolor)
    fig.update_yaxes(zeroline=True, showgrid=False, color=titlecolor, 
                     title_font_color=titlecolor, tickfont=dict(size=11), title_text='')
    fig.update_traces(hovertemplate='%{x:.0f} (mW)', texttemplate='%{x:.0f} mW', 
                      textfont=dict(size=11))
    
    fig.update_layout(autosize=True, hovermode=False, legend_title_text='',
                      showlegend=False, font=dict(family="SF Pro Display, Roboto, Droid Sans, Arial"),
                      title={
                          'text': "<b>Average Power Consumption</b> <br> <sup> OPT 1.3B | Flan-T5 783M | Bloom 1.1B </sup>",
                          'y': 0.90,
                          'x': 0.54,
                          'xanchor': 'center',
                          'yanchor': 'top',
                          'font': dict(size=18, color=titlecolor)},
                      margin=dict(r=30, b=15, t=60),
                      margin_pad=10,
                      modebar=dict(orientation='v'),
                      plot_bgcolor=bgcolor,
                      paper_bgcolor=bgcolor,
                     )

    # Save the figure as SVG and show
    fig.write_image("./power-average-bar.svg")  # Save as svg
    fig.show()

# Example usage with your DataFrame
print(dfPower.head())
plot_average_power_bar(dfPower, "Power Consumption")


   Efficiency Cluster  Performance Cluster  CPU Power  GPU Power  ANE Power  \
0                1287                  747        153         24          0   
1                1050                  611         39         24          0   
2                1094                  652         86          6          0   
3                1227                  618        106          6          0   
4                1052                  608         36         24          0   

   Combined Power  time   Model Output Length Input Length  
0             177     1  FlanT5           100         long  
1              45     2  FlanT5           100         long  
2             110     3  FlanT5           100         long  
3             112     4  FlanT5           100         long  
4              42     5  FlanT5           100         long  


In [16]:
def plot_average_energy_bar(dataframe, title):
    # Calculate Energy (J) by multiplying Combined Power (mW) by time (s) and converting to joules (1 mW = 0.001 J)
    dataframe['Energy (J)'] = (dataframe['Combined Power'] * dataframe['time']) * 0.001

    # Calculate the average Energy (J) for each model
    barDf = dataframe.groupby('Model')['Energy (J)'].mean().reset_index()

    # Create a horizontal bar chart
    fig = px.bar(barDf, y='Model', x='Energy (J)', template='plotly_white',
                 orientation='h', hover_name='Model', 
                 width=700, height=250, 
                 color_discrete_sequence=[green],
                 labels={"Energy (J)": "Energy Consumption (J)"})

    fig.update_xaxes(zeroline=True, title_font=dict(size=12), color=titlecolor,
                     title_font_color=titlecolor, tickfont=dict(size=9), 
                     gridcolor=graycolor, zerolinecolor=graycolor)
    fig.update_yaxes(zeroline=True, showgrid=False, color=titlecolor, 
                     title_font_color=titlecolor, tickfont=dict(size=11), title_text='')
    fig.update_traces(hovertemplate='%{x:.4f} (J)', texttemplate='%{x:.4f} J', 
                      textfont=dict(size=11))
    
    fig.update_layout(autosize=True, hovermode=False, legend_title_text='',
                      showlegend=False, font=dict(family="SF Pro Display, Roboto, Droid Sans, Arial"),
                      title={
                          'text': "<b>Average Energy Consumption</b> <br> <sup> OPT 1.3B | Flan-T5 783M | Bloom 1.1B </sup>",
                          'y': 0.90,
                          'x': 0.54,
                          'xanchor': 'center',
                          'yanchor': 'top',
                          'font': dict(size=18, color=titlecolor)},
                      margin=dict(r=30, b=15, t=60),
                      margin_pad=10,
                      modebar=dict(orientation='v'),
                      plot_bgcolor=bgcolor,
                      paper_bgcolor=bgcolor,
                     )

    # Save the figure as SVG and show
    fig.write_image("./energy-average-bar.png")  # Save as svg
    fig.show()

# Example usage with combined_power_time_summary
print(combined_power_time_summary.head())
plot_average_energy_bar(combined_power_time_summary, "Energy Consumption")



   Model Output Length Input Length  Combined Power    time   Energy (J)
0  Bloom           100         long     1982.306497   708.5  1404.464153
1  Bloom           100        short     2003.937410   695.5  1393.738469
2  Bloom           200         long     2182.136155  1631.0  3559.064068
3  Bloom           200        short     2183.553411  1554.5  3394.333777
4  Bloom            50         long     1811.266667   375.5   680.130633


In [60]:
# Group the dataframe by Model and Input/Output Length, taking the mean of Combined Power for the other settings
power_avg_output = (dfPower.groupby(['Model', 'Output Length'])['Combined Power']
                   .mean()
                   .reset_index())

power_avg_input = (dfPower.groupby(['Model', 'Input Length'])['Combined Power']
                   .mean()
                   .reset_index())

# Plot the average power consumption for different output lengths of each model
def plot_average_power_output(dataframe, title):

    if title == "Input":
        color_discrete_map={"0-60 tokens": "#73A4FF", "60-110 tokens": "#66D8BA"}
        length_labels = {'short': '0-60 tokens', 'long': '60-110 tokens'}
        category_order = {"Input Length": ["0-60 tokens", "60-110 tokens"]}
    elif title == "Output":
        color_discrete_map={"50 tokens": "#FF715A","100 tokens": "#73A4FF", "200 tokens": "#66D8BA"}
        length_labels = {'50': '50 tokens', '100': '100 tokens', '200': '200 tokens'}
        category_order = {"Output Length": ["50 tokens", "100 tokens", "200 tokens"]}
    
    # Map the new labels to the appropriate column in the dataframe
    dataframe[f'{title} Length'] = dataframe[f'{title} Length'].map(length_labels)

    fig = px.bar(dataframe, 
                 x='Model', 
                 y='Combined Power', 
                 color=f'{title} Length',  # Different bars for each output length
                 barmode='group',  # Group bars by model
                 title=title, 
                 color_discrete_map=color_discrete_map,
                 category_orders=category_order,
                 labels={'Combined Power': 'Mean Average Combined Power (mW)', f'{title} Length': f'{title} Length'},
                 template='plotly_dark', 
                 width=800, 
                 height=500,
                 text_auto=True)

    fig.update_yaxes(title_font = dict(size=12), title_font_color = titlecolor, color=titlecolor,  tickfont = dict(size = 9), gridcolor=graycolor, zerolinecolor = graycolor) #, range=[0, 1100])
    fig.update_xaxes(zeroline = True, showgrid=False, color=titlecolor, title_font_color = titlecolor, tickfont = dict(size = 11), title_text='')
    fig.update_traces(textfont= dict(size=8), width=[0.16, 0.16, 0.16])
    fig.update_layout(autosize=True, hovermode=False, 
                      legend_title_text='', 
                      legend=dict(orientation="h", yanchor="bottom", y=1, 
                                  xanchor="center", x=0.5, title=f"{title} Length"),
                      font=dict(family="SF Pro Display, Roboto, Droid Sans, Arial"),
                      title={
                          'text': f"<b>Mean Average Power Consumption by {title} Length</b> <br>",
                          'y': 0.93,
                          'x': 0.54,
                          'xanchor': 'center',
                          'yanchor': 'top',
                          'font': dict(size=18, color=titlecolor)},
                      margin=dict(r=30, b=0, t=80), margin_pad=10,
                      modebar=dict(orientation='v'),
                      plot_bgcolor=bgcolor,
                      paper_bgcolor=bgcolor)

    fig.write_image(f"./power-average-bar-{title}.png")  # Save as PNG
    fig.show()

# Call the function to plot for all models
plot_average_power_output(power_avg_output, "Output")
plot_average_power_output(power_avg_input, "Input")



In [14]:
# Group the combined_power_time_summary dataframe by Model and Output/Input Length, taking the mean of Combined Power
power_avg_output = (combined_power_time_summary.groupby(['Model', 'Output Length'])['Combined Power']
                   .mean()
                   .reset_index())

power_avg_input = (combined_power_time_summary.groupby(['Model', 'Input Length'])['Combined Power']
                   .mean()
                   .reset_index())

# Plot the average power consumption for different output lengths of each model
def plot_average_power_output(dataframe, title):

    if title == "Input":
        color_discrete_map={"0-60 tokens": "#73A4FF", "60-110 tokens": "#66D8BA"}
        length_labels = {'short': '0-60 tokens', 'long': '60-110 tokens'}
        category_order = {"Input Length": ["0-60 tokens", "60-110 tokens"]}
    elif title == "Output":
        color_discrete_map={"50 tokens": "#FF715A","100 tokens": "#73A4FF", "200 tokens": "#66D8BA"}
        length_labels = {'50': '50 tokens', '100': '100 tokens', '200': '200 tokens'}
        category_order = {"Output Length": ["50 tokens", "100 tokens", "200 tokens"]}
    
    # Map the new labels to the appropriate column in the dataframe
    dataframe[f'{title} Length'] = dataframe[f'{title} Length'].map(length_labels)

    fig = px.bar(dataframe, 
                 x='Model', 
                 y='Combined Power', 
                 color=f'{title} Length',  # Different bars for each output length
                 barmode='group',  # Group bars by model
                 title=title, 
                 color_discrete_map=color_discrete_map,
                 category_orders=category_order,
                 labels={'Combined Power': 'Mean Average Combined Power (mW)', f'{title} Length': f'{title} Length'},
                 template='plotly_dark', 
                 width=800, 
                 height=500,
                 text_auto=True)

    fig.update_yaxes(title_font = dict(size=12), title_font_color = titlecolor, color=titlecolor,  tickfont = dict(size = 9), gridcolor=graycolor, zerolinecolor = graycolor)
    fig.update_xaxes(zeroline = True, showgrid=False, color=titlecolor, title_font_color = titlecolor, tickfont = dict(size = 11), title_text='')
    fig.update_traces(textfont= dict(size=8), width=[0.16, 0.16, 0.16])
    fig.update_layout(autosize=True, hovermode=False, 
                      legend_title_text='', 
                      legend=dict(orientation="h", yanchor="bottom", y=1, 
                                  xanchor="center", x=0.5, title=f"{title} Length"),
                      font=dict(family="SF Pro Display, Roboto, Droid Sans, Arial"),
                      title={
                          'text': f"<b>Mean Average Power Consumption by {title} Length</b> <br>",
                          'y': 0.93,
                          'x': 0.54,
                          'xanchor': 'center',
                          'yanchor': 'top',
                          'font': dict(size=18, color=titlecolor)},
                      margin=dict(r=30, b=0, t=80), margin_pad=10,
                      modebar=dict(orientation='v'),
                      plot_bgcolor=bgcolor,
                      paper_bgcolor=bgcolor)

    fig.write_image(f"./power-average-bar-{title}.png")  # Save as PNG
    fig.show()

# Call the function to plot for all models
plot_average_power_output(power_avg_output, "Output")
plot_average_power_output(power_avg_input, "Input")


In [15]:
# Calculate the energy consumption in Joules (J)
combined_power_time_summary['Energy (J)'] = (combined_power_time_summary['Combined Power'] / 1000) * combined_power_time_summary['time']

# Group the combined_power_time_summary dataframe by Model and Output/Input Length, taking the mean of Energy
energy_avg_output = (combined_power_time_summary.groupby(['Model', 'Output Length'])['Energy (J)']
                   .mean()
                   .reset_index())

energy_avg_input = (combined_power_time_summary.groupby(['Model', 'Input Length'])['Energy (J)']
                   .mean()
                   .reset_index())

# Plot the average energy consumption for different output lengths of each model
def plot_average_energy_output(dataframe, title):

    if title == "Input":
        color_discrete_map={"0-60 tokens": "#73A4FF", "60-110 tokens": "#66D8BA"}
        length_labels = {'short': '0-60 tokens', 'long': '60-110 tokens'}
        category_order = {"Input Length": ["0-60 tokens", "60-110 tokens"]}
    elif title == "Output":
        color_discrete_map={"50 tokens": "#FF715A","100 tokens": "#73A4FF", "200 tokens": "#66D8BA"}
        length_labels = {'50': '50 tokens', '100': '100 tokens', '200': '200 tokens'}
        category_order = {"Output Length": ["50 tokens", "100 tokens", "200 tokens"]}
    
    # Map the new labels to the appropriate column in the dataframe
    dataframe[f'{title} Length'] = dataframe[f'{title} Length'].map(length_labels)

    fig = px.bar(dataframe, 
                 x='Model', 
                 y='Energy (J)',  # Use Energy in Joules for the y-axis
                 color=f'{title} Length',  # Different bars for each output length
                 barmode='group',  # Group bars by model
                 title=title, 
                 color_discrete_map=color_discrete_map,
                 category_orders=category_order,
                 labels={'Energy (J)': 'Mean Average Energy Consumption (J)', f'{title} Length': f'{title} Length'},
                 template='plotly_dark', 
                 width=800, 
                 height=500,
                 text_auto=True)

    fig.update_yaxes(title_font=dict(size=12), title_font_color=titlecolor, color=titlecolor, tickfont=dict(size=9), gridcolor=graycolor, zerolinecolor=graycolor)
    fig.update_xaxes(zeroline=True, showgrid=False, color=titlecolor, title_font_color=titlecolor, tickfont=dict(size=11), title_text='')
    fig.update_traces(textfont=dict(size=8), width=[0.16, 0.16, 0.16])
    fig.update_layout(autosize=True, hovermode=False, 
                      legend_title_text='', 
                      legend=dict(orientation="h", yanchor="bottom", y=1, 
                                  xanchor="center", x=0.5, title=f"{title} Length"),
                      font=dict(family="SF Pro Display, Roboto, Droid Sans, Arial"),
                      title={
                          'text': f"<b>Mean Average Energy Consumption by {title} Length</b> <br>",
                          'y': 0.93,
                          'x': 0.54,
                          'xanchor': 'center',
                          'yanchor': 'top',
                          'font': dict(size=18, color=titlecolor)},
                      margin=dict(r=30, b=0, t=80), margin_pad=10,
                      modebar=dict(orientation='v'),
                      plot_bgcolor=bgcolor,
                      paper_bgcolor=bgcolor)

    fig.write_image(f"./energy-average-bar-{title}.png")  # Save as PNG
    fig.show()

# Call the function to plot for all models
plot_average_energy_output(energy_avg_output, "Output")
plot_average_energy_output(energy_avg_input, "Input")


In [61]:
def plot_power_over_time(dataframe, title):
    # Create the area plot with facets for each model
    fig = px.area(dfPower, x='time', y=['Combined Power'], template='plotly_dark', 
    width = 1700, height = 350, facet_col='Model', line_shape="spline", facet_col_wrap=5,
    labels={"value": "Power Consumption (mW)", "time": "Time (s)"}, color_discrete_map={"Combined Power": "#66D8BA"},
    category_orders={"Model": ["Bloom", "FlanT5", "OPT"]})  # Ensure correct model ordering
    
    # Remove the facet annotation texts ("Model=Bloom", etc.)
    fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
    
    # Customize annotations' font
    for annotation in fig['layout']['annotations']:
        annotation['font'] = dict(family="SF Pro Display, Roboto, Droid Sans, Arial", size=11)
    
    # Update y-axis settings
    fig.update_yaxes(type='linear', title_font=dict(size=12), color=titlecolor, title_font_color=titlecolor, 
                     tickfont=dict(size=9), gridcolor=graycolor, zerolinecolor=graycolor)
    
    # Update x-axis settings
    fig.update_xaxes(showgrid=False, title_font=dict(size=10), color=titlecolor, title_font_color=titlecolor, 
                     tickfont=dict(size=9))
    
    # Customize the hover template and trace smoothing
    fig.update_traces(hovertemplate='%{y} (mW)', line_smoothing=1.3)
    
    # Dynamically adjust the x-axis range for each model using Plotly's facet mechanism
    for i, model in enumerate(["Bloom", "FlanT5", "OPT"]):
        # Get the maximum 'time' for the current model
        max_time = dataframe[dataframe['Model'] == model]['time'].max()
        
        # Update the x-axis range for the subplot corresponding to the current model
        fig.update_xaxes(matches=None, range=[0, max_time], row=1, col=i+1)  # Using col as the facet position
        
    # Customize layout and title
    fig.update_layout(autosize=True, hovermode="x", showlegend=False, 
                      font=dict(family="SF Pro Display, Roboto, Droid Sans, Arial"),
                      title={
                          'text': "<b>Combined Power Consumption over Time</b> <br> <sup> OPT 1.3B | Flan-T5 783M | Bloom 1.1B </sup>",
                          'y': 0.92,
                          'x': 0.54,
                          'xanchor': 'center',
                          'yanchor': 'top',
                          'font': dict(size=18, color=titlecolor)},
                      margin=dict(r=50, t=80), margin_pad=10,
                      modebar=dict(orientation='v'), plot_bgcolor=bgcolor, paper_bgcolor=bgcolor)

    # Save the figure as a PNG and show it
    fig.write_image("./power-combined-time.png")
    fig.show()

# Call the function to generate the plot
plot_power_over_time(dfPower, "title")



## Statistics

In [ ]:
import numpy as np
from scipy import stats

# Energy consumption data for each model
opt_consumption = 2080  # in mW
flant5_consumption = 1482  # in mW
bloom_consumption = 1427  # in mW

# Prepare the data for paired t-tests
data = np.array([
    [opt_consumption, flant5_consumption],
    [opt_consumption, bloom_consumption],
    [flant5_consumption, bloom_consumption],
])

# Perform paired t-tests
t_results = {
    'OPT vs Flan-T5': stats.ttest_rel(data[0], data[1]),
    'OPT vs BLOOM': stats.ttest_rel(data[0], data[2]),
    'Flan-T5 vs BLOOM': stats.ttest_rel(data[1], data[2]),
}

t_results

In [ ]:
import pandas as pd
import pingouin as pg
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd, MultiComparison

# Define the data
data = {
    'Model': ['Bloom'] * 6 + ['FlanT5'] * 6 + ['OPT'] * 6,
    'Output Length': [100, 100, 200, 200, 50, 50] * 3,
    'Input Length': ['long', 'short'] * 3 + ['long', 'short'] * 3 + ['long', 'short'] * 3,
    'Combined Power': [
        1982.306497, 2003.937410, 2182.136155, 2183.553411, 1811.266667, 1786.673130,
        1465.607619, 1446.385396, 1597.229448, 1535.638519, 1334.101990, 1369.100000,
        1205.122982, 1301.159158, 1720.379387, 1410.372432, 995.001415, 1137.443656
    ]
}

# Create DataFrame
df = pd.DataFrame(data)
# Renaming the columns in the DataFrame
df.rename(columns={'Input Length': 'Input', 'Output Length': 'Output', 'Combined Power': 'Power'}, inplace=True)
print(df)

# RQ1: Two-way ANOVA using Pingouin
anova2 = pg.anova(dv='Power', between=['Model', 'Input'], data=df)
print("RQ1: Two-Way ANOVA Results")
print(anova2)


anova2 = ols('Power ~ C(Model) + C(Input) + C(Model):C(Input)', data=df).fit()
anova_table = sm.stats.anova_lm(anova2, typ=2)

print("RQ1: Two-Way ANOVA Results")
print(anova_table)

# Post-hoc test (Tukey's HSD) using pairwise_ttests with Bonferroni correction
posthoc = pg.pairwise_ttests(dv='Power', between=['Model', 'Input'], data=df, padjust='bonf')
print("\nPost-hoc Test (Bonferroni-corrected) Results:")
print(posthoc)

In [ ]:
# Step 1: Average Power for each Model and Output Length (ignoring Input for this ANOVA)
df_summary = df.groupby(['Model', 'Output'])['Power'].mean().reset_index()

# Step 2: Conduct repeated measures ANOVA
rm_anova = pg.rm_anova(dv='Power', within='Output', subject='Model', data=df_summary)
print("Repeated Measures ANOVA Results:")
print(rm_anova)

# Step 3: Post-hoc tests with Bonferroni correction
posthoc = pg.pairwise_ttests(dv='Power', within='Output', subject='Model', padjust='bonf', data=df_summary)
print("\nBonferroni Post-hoc Test Results:")
print(posthoc)